## Project: **Finding Lane Lines on the Road** 

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

ImportError: DLL load failed: The specified module could not be found.

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images    
`cv2.cvtColor()` to grayscale or change color    
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

In [36]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=15):
    """
    This function draws `lines` with `color` and `thickness`.    
    
    Lines are drawn on the image inplace (mutates the image).
    
    Equations
    m = (y2-y1)/(x2-x1)
    y = mx + b
    b = y - mx = y - x(y2-y1)/(x2-x1)
    """
    if lines is None:
        return img
    
    # separate line segments into right and left
    left_lines = []
    right_lines = []
    width, height, depth = img.shape
    for line in lines:
        for line_list in line:
            x1,y1,x2,y2 = line_list
            m = float(y2-y1)/(x2-x1)
            intercept = calc_intercept(x1,y1,x2,y2, m)
            
            y1 = int(width * 0.70)
            x1 = int((y1 - intercept) / m)
            y2 = int(width)
            x2 = int((y2 - intercept) / m)
                    
            if m < 0: #left lanes
                left_lines.append(line)
                cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), thickness)
            else: #right lanes (m > 0)
                right_lines.append(line)
                cv2.line(img, (x1, y1), (x2, y2), (255, 0, 175), thickness)

              
def calc_intercept(x1,y1,x2,y2, slope):
    """ Calculates the intercept of line cy = slope * cx + intercept 
        where cx, cy in the center of the line
    """
    cx = (x1 + x2) / 2
    cy = (y1 + y2) / 2
    return cy - slope * cx



def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

img = mpimg.imread('test_images/solidWhiteCurve.jpg')
pipeline_process(img)

## Build a Lane Finding Pipeline



#### Pipeline Initiate Data

In [37]:
kernel_size = 3
low_threshold = 64 # Define our parameters for Canny and apply
high_threshold = 192 # Define our parameters for Canny and apply


##### Define the Hough transform parameters
rho = 1 # distance resolution in pixels of the Hough grid
theta = np.pi/180 # angular resolution in radians of the Hough grid
threshold = 50     # minimum number of votes (intersections in Hough grid cell)
min_line_len = 100 #minimum number of pixels making up a line
max_line_gap = 160    # maximum gap in pixels between connectable line segments

#### Building The Image & Video Processing Pipeline function

In [38]:
def build_pipeline(image):
    h, w, _ = image.shape
    vertices = np.array([[w*0.125, h], [w*0.475, h*0.59], [w*0.52, h*0.59], [w*0.95, h]], dtype=np.int32)
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    yellow = cv2.inRange(hsv, (20, 80, 80), (25, 255, 255))
    white = cv2.inRange(hsv, (0, 0, 180), (255, 25, 255))
    gray = cv2.bitwise_or(yellow, white)
    edges = canny(gray, low_threshold, high_threshold)
    roi = region_of_interest(edges, [vertices])
    roi2 = region_of_interest(gray, [vertices])
    lines = hough_lines(roi2, rho, theta, threshold, min_line_len, max_line_gap)
    
    #Scanner Checker
    if(scanner==True):     
        image = region_of_interest(image, [vertices]) #scanner shape
        scanner_edges = canny(image, low_threshold, high_threshold)
        scanner_lines = hough_lines(scanner_edges, rho, theta, threshold, min_line_len, max_line_gap)
        
        
    result = weighted_img(image, lines, 0.9, 0.9)
    return result

## Test on Videos
We can test our solution on provided videos:   
`solidWhiteRight.mp4`   
`solidYellowLeft.mp4`    
`challenge.mp4`

In [39]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML, display

#### Video Pipeline Generator & Video Loader Function

In [40]:
def video_generator(input,output,filter):
    clip = VideoFileClip(input)
    white_clip = clip.fl_image(filter) #NOTE: this function expects color images!!
    %time white_clip.write_videofile(output, audio=False)
    

def video_loader(header_text,ori_video,video,scanner_shape):
    display(HTML('<h2 style="text-align:center;">' + header_text + '</h2>' +
                 '<video width="320" height="180" controls><source src="' + ori_video + '"></video>' +
                 '<video width="320" height="180" controls><source src="' + video + '"></video>' +
                 '<video width="320" height="180" controls><source src="' + scanner_shape + '"></video>'))

#### Video Data

In [41]:
video_data = [['White Challenge','solidWhiteRight.mp4','white.mp4','whiteshape.mp4'],
             ['Yellow Challenge','solidYellowLeft.mp4','yellow.mp4','yellowshape.mp4'],
             ['Extra Challenge','challenge.mp4','extra.mp4','extrashape.mp4']] # Optional Challenge

#Test Code , will remove soon
scanner = False
video_generator(video_data[2][1],video_data[2][2],build_pipeline)

#### Generate Pipeline on Video And Display All Video

In [42]:
for i in video_data:    
    scanner = False
    video_generator(i[1],i[2],build_pipeline)
    scanner = True
    video_generator(i[1],i[3],build_pipeline)
    video_loader(i[0],i[1],i[2],i[3])

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4






  0%|          | 0/222 [00:00<?, ?it/s]



  5%|▌         | 12/222 [00:00<00:01, 112.70it/s]



 11%|█         | 24/222 [00:00<00:01, 112.38it/s]



 16%|█▌        | 36/222 [00:00<00:01, 112.63it/s]



 21%|██        | 47/222 [00:00<00:01, 104.33it/s]



 25%|██▍       | 55/222 [00:00<00:01, 95.11it/s] 



 29%|██▉       | 64/222 [00:00<00:01, 91.08it/s]



 32%|███▏      | 72/222 [00:00<00:01, 86.83it/s]



 36%|███▋      | 81/222 [00:00<00:01, 87.31it/s]



 41%|████      | 90/222 [00:00<00:01, 85.71it/s]



 45%|████▍     | 99/222 [00:01<00:01, 84.97it/s]



 49%|████▊     | 108/222 [00:01<00:01, 85.12it/s]



 53%|█████▎    | 117/222 [00:01<00:01, 84.48it/s]



 57%|█████▋    | 127/222 [00:01<00:01, 86.14it/s]



 61%|██████▏   | 136/222 [00:01<00:00, 86.57it/s]



 65%|██████▌   | 145/222 [00:01<00:00, 85.67it/s]



 69%|██████▉   | 154/222 [00:01<00:00, 86.09it/s]



 73%|███████▎  | 163/222 [00:01<00:00, 87.14it/s]



 77%|███████▋  | 172/222 [00:01<00:00, 85.80it/s]



 82%

[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 3.13 s, sys: 188 ms, total: 3.32 s
Wall time: 2.83 s
[MoviePy] >>>> Building video whiteshape.mp4
[MoviePy] Writing video whiteshape.mp4






  0%|          | 0/222 [00:00<?, ?it/s]


 42%|████▏     | 287/682 [00:19<00:27, 14.51it/s]



  4%|▎         | 8/222 [00:00<00:03, 70.13it/s]



  7%|▋         | 15/222 [00:00<00:02, 70.09it/s]



 10%|█         | 23/222 [00:00<00:02, 70.35it/s]



 14%|█▎        | 30/222 [00:00<00:02, 70.14it/s]



 17%|█▋        | 38/222 [00:00<00:02, 70.84it/s]



 20%|██        | 45/222 [00:00<00:02, 69.10it/s]



 23%|██▎       | 52/222 [00:00<00:02, 66.14it/s]



 27%|██▋       | 59/222 [00:00<00:02, 63.41it/s]



 30%|██▉       | 66/222 [00:00<00:02, 62.95it/s]



 33%|███▎      | 73/222 [00:01<00:02, 63.57it/s]



 36%|███▌      | 80/222 [00:01<00:02, 64.66it/s]



 39%|███▉      | 87/222 [00:01<00:02, 65.38it/s]



 42%|████▏     | 94/222 [00:01<00:02, 63.83it/s]



 45%|████▌     | 101/222 [00:01<00:01, 64.38it/s]



 49%|████▊     | 108/222 [00:01<00:01, 65.74it/s]



 52%|█████▏    | 115/222 [00:01<00:01, 65.71it/s]



 55%|█████▍    | 122/222 [00:01<00:01, 65.09it/s]



 58%|█████▊   

[MoviePy] Done.
[MoviePy] >>>> Video ready: whiteshape.mp4 

CPU times: user 5.2 s, sys: 356 ms, total: 5.55 s
Wall time: 3.46 s


[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4






  0%|          | 0/682 [00:00<?, ?it/s]



  2%|▏         | 12/682 [00:00<00:05, 114.43it/s]



  4%|▎         | 24/682 [00:00<00:05, 114.00it/s]



  5%|▌         | 36/682 [00:00<00:05, 113.11it/s]



  7%|▋         | 47/682 [00:00<00:06, 104.43it/s]



  8%|▊         | 56/682 [00:00<00:06, 98.00it/s] 



 10%|▉         | 65/682 [00:00<00:06, 92.02it/s]



 11%|█         | 74/682 [00:00<00:06, 90.77it/s]



 12%|█▏        | 83/682 [00:00<00:06, 87.84it/s]



 14%|█▎        | 93/682 [00:00<00:06, 88.26it/s]



 15%|█▍        | 102/682 [00:01<00:06, 87.35it/s]



 16%|█▋        | 111/682 [00:01<00:06, 85.49it/s]



 18%|█▊        | 120/682 [00:01<00:06, 82.65it/s]



 19%|█▉        | 129/682 [00:01<00:06, 81.40it/s]



 20%|██        | 138/682 [00:01<00:06, 82.82it/s]



 22%|██▏       | 147/682 [00:01<00:06, 84.48it/s]



 23%|██▎       | 156/682 [00:01<00:06, 84.59it/s]



 24%|██▍       | 165/682 [00:01<00:06, 84.80it/s]



 26%|██▌       | 174/682 [00:01<00:06, 84.62it/s]



 27

[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 10.3 s, sys: 504 ms, total: 10.8 s
Wall time: 8.34 s
[MoviePy] >>>> Building video yellowshape.mp4
[MoviePy] Writing video yellowshape.mp4






  0%|          | 0/682 [00:00<?, ?it/s]



  1%|          | 7/682 [00:00<00:09, 69.69it/s]



  2%|▏         | 14/682 [00:00<00:09, 69.56it/s]



  3%|▎         | 22/682 [00:00<00:09, 70.26it/s]



  4%|▍         | 29/682 [00:00<00:09, 68.90it/s]



  5%|▌         | 36/682 [00:00<00:09, 69.01it/s]



  6%|▋         | 43/682 [00:00<00:09, 68.74it/s]



  7%|▋         | 50/682 [00:00<00:09, 67.97it/s]



  8%|▊         | 57/682 [00:00<00:09, 66.85it/s]



  9%|▉         | 64/682 [00:00<00:09, 66.35it/s]



 10%|█         | 71/682 [00:01<00:09, 65.76it/s]



 11%|█▏        | 78/682 [00:01<00:09, 65.41it/s]



 12%|█▏        | 85/682 [00:01<00:09, 66.01it/s]



 13%|█▎        | 92/682 [00:01<00:09, 65.37it/s]



 15%|█▍        | 99/682 [00:01<00:08, 65.37it/s]



 16%|█▌        | 106/682 [00:01<00:08, 64.31it/s]



 17%|█▋        | 113/682 [00:01<00:08, 63.89it/s]



 18%|█▊        | 120/682 [00:01<00:08, 64.23it/s]



 19%|█▊        | 127/682 [00:01<00:08, 65.01it/s]



 20%|█▉       

OverflowError: cannot convert float infinity to integer

## Test Images
**Test on images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

#### Process Of Building Pipeline In Image

In [ ]:
def showimg(subplace, title, _img):
    plt.subplot(*subplace)
    plt.title(title)
    if len(_img.shape) == 3:
        plt.imshow(_img)
    else:
        plt.imshow(_img, cmap='gray')
        
def pipeline_process(image):
    h, w, _ = image.shape
    vertices = np.array([[w*0.125, h], [w*0.475, h*0.59], [w*0.52, h*0.59], [w*0.95, h]], dtype=np.int32)
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    yellow = cv2.inRange(hsv, (20, 80, 80), (25, 255, 255))
    white = cv2.inRange(hsv, (0, 0, 180), (255, 25, 255))
    gray = cv2.bitwise_or(yellow, white)
    edges = canny(gray, low_threshold, high_threshold)
    roi = region_of_interest(edges, [vertices])
    roi2 = region_of_interest(gray, [vertices])
    lines = hough_lines(roi2, rho, theta, 50, min_line_len, max_line_gap)
    result = weighted_img(image, lines, 0.9, 0.9)

    #Scanner checker
    scanner_shape = region_of_interest(image, [vertices])
    scanner_edges = canny(scanner_shape, low_threshold, high_threshold)
    scanner_lines = hough_lines(scanner_edges, rho, theta, threshold, min_line_len, max_line_gap)
    scanner = weighted_img(scanner_shape, lines, 0.9, 0.9)

    plt.figure(figsize=[20, 13])    
    for i, img in enumerate(['scanner_shape', 'scanner_edges', 'scanner_lines', 'scanner']):
        showimg((2, 2, i+1), img, eval(img))
        
    plt.figure(figsize=[20, 13])
    for i, img in enumerate(['image', 'yellow', 'white', 'gray', 'edges', 'roi', 'roi2', 'lines','result']):
        showimg((3, 3, i+1), img, eval(img))


#### Generate Pipeline On Images And Display All The Process

In [ ]:
for test_image in os.listdir("test_images/"):
    img = mpimg.imread('test_images/' + test_image)
    pipeline_process(img)